In [6]:
import pandas as pd
import numpy as np

import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold,GroupKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import gc
import re
from sklearn.metrics import roc_auc_score

import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import polars as pl
from pathlib import Path
from glob import glob
import json
import joblib
import warnings
warnings.filterwarnings("ignore")




In [7]:
with open('../IND-WhoIsWho/pid_to_info_all.json', 'r') as file:
    pid = json.load(file)

In [9]:

train = pd.read_feather('data/train.feather')
valid = pd.read_feather('data/valid.feather')
test = pd.read_feather('data/test.feather')

train['dataset_mode'] = 0
valid['dataset_mode'] = 1
test['dataset_mode'] = 2

data = pd.concat([train,valid,test]).reset_index(drop = True)

data['id'] = data.index

In [10]:

df = pd.read_feather('temp_data/autherID_authors.feather')
df = df[df['flag']!=1].reset_index(drop = True)
df = df.merge(data[['autherID','PID','id']],on = ['autherID','PID'],how = 'left')
t_df = df[df['auther_authors_name_count']>=2].reset_index(drop = True)
#t_df = t_df[t_df['label'].notna()].reset_index(drop = True)
t_df = t_df.groupby(['autherID','authors_name'])['PID'].agg(list).reset_index()
t_df = t_df[t_df['PID'].apply(lambda x:len(x)>1)].reset_index(drop = True)
t_df['PID'] = t_df['PID'].apply(lambda x:'  '.join(sorted(x)))
t_df = t_df.drop_duplicates(subset = ['PID'],keep = 'first').reset_index(drop = True)
t_df['PID'] = t_df['PID'].apply(lambda x:x.split('  ' ))
t_df1 = t_df.copy()


In [11]:
data['keywords'] = data['PID'].apply(lambda x:pid[x]['keywords'])

ans = []
for auther,keywords,PID, in tqdm(data[['autherID','keywords','PID']].values):
    if len(keywords)>0 and type(keywords)== list and keywords[0] !='null':
        for i in range(len(keywords)):
            ans.append([auther,keywords[i],PID])
            
df = pd.DataFrame(ans,columns = ['autherID','keyword','PID'])
df['auther_keyword_count'] = df.groupby(['autherID','keyword'])['PID'].transform('count')


100%|██████████| 2957/2957 [00:00<00:00, 271040.82it/s]


In [12]:
data['keywords'] = data['PID'].apply(lambda x:pid[x]['keywords'])

ans = []
for auther,keywords,PID, in tqdm(data[['autherID','keywords','PID']].values):
    if len(keywords)>0 and type(keywords)== list and keywords[0] !='null':
        for i in range(len(keywords)):
            ans.append([auther,keywords[i],PID])
            
df = pd.DataFrame(ans,columns = ['autherID','keyword','PID'])
df['auther_keyword_count'] = df.groupby(['autherID','keyword'])['PID'].transform('count')

t_df = df[df['auther_keyword_count']>=2].reset_index(drop = True)
#t_df = t_df[t_df['label'].notna()].reset_index(drop = True)
t_df = t_df.groupby(['autherID','keyword'])['PID'].agg(list).reset_index()
t_df = t_df[t_df['PID'].apply(lambda x:len(x)>1)].reset_index(drop = True)
t_df['PID'] = t_df['PID'].apply(lambda x:'  '.join(sorted(x)))
t_df = t_df.drop_duplicates(subset = ['PID'],keep = 'first').reset_index(drop = True)
t_df['PID'] = t_df['PID'].apply(lambda x:x.split('  ' ))
t_df2 = t_df.copy()

100%|██████████| 2957/2957 [00:00<00:00, 308199.32it/s]


In [14]:
def text_clean(x,mode = 1):
    x = x.lower()
    for f in list('-?=～—/_？）￥:#\\\'.》”^>$]}|+)、（&{`《,(%!“<’"】；【‘~*@…：，。[;') :
        x = x.replace(f,'')
        
    if mode == 'venue':
        number_pattern = r'\d+'
        x = re.sub(number_pattern, '', x)
    
    
    for i in range(3):
        x = x.replace('  ',' ')
    return x




data = pd.concat([train,valid,test]).reset_index(drop = True)


data['pid_title'] = data['PID'].apply(lambda x:pid[x]['title'])
data['abstract'] = data['PID'].apply(lambda x:pid[x]['abstract'])
data['keywords'] = data['PID'].apply(lambda x:pid[x]['keywords'])
data['venue'] = data['PID'].apply(lambda x:pid[x]['venue'])
data['year'] = data['PID'].apply(lambda x:pid[x]['year'])
data['authors'] = data['PID'].apply(lambda x:pid[x]['authors'])

data['venue'] = data['venue'].fillna('')
data['venue'] = data['venue'].apply(lambda x:text_clean(x,mode = 'venue'))
"""
data['venue_id'] = data['venue'].map(dict(temp.values))

data['venue'] = data['venue'].apply(lambda x:x.lower())
temp = data.groupby(['autherID','venue_id'])['label','year','PID'].agg(list).reset_index()
temp = temp[temp['label'].apply(lambda x:len(x) >1)]
#temp = temp[temp['venue'] != '']
t_df3 = temp.copy()
"""
data['venue'] = data['venue'].apply(lambda x:x.lower())
temp = data.groupby(['autherID','venue'])['label','year','PID'].agg(list).reset_index()
temp = temp[temp['label'].apply(lambda x:len(x) >1)]
temp = temp[temp['venue'] != '']
t_df3 = temp.copy()



In [15]:

df = pd.read_feather('temp_data/autherID_authors.feather')
df = df[df['authors_org'] != ''].reset_index(drop= True)
df = df[df['flag'] ==1].reset_index(drop= True)

df = df.sort_values(['autherID','auther_authors_name_count'],ascending = False).reset_index(drop = True)
df = df.groupby(['autherID','PID']).head(1).reset_index(drop = True)

df = df.merge(data[['autherID','PID','label','year']],on = ['autherID','PID'],how = 'left')

temp = df.groupby(['autherID','authors_org'])['label','year','PID'].agg(list).reset_index()
temp = temp[temp['label'].apply(lambda x:len(x)>1)]
t_df4 = temp.copy()

In [16]:
def get_split_year(t_df1):
    
    t_dict = dict(zip(data['autherID'] +'_'+ data['PID'].to_list(),data['year'].to_list()))
    t_df1['year'] = t_df1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)

    t_dict = dict(zip(data['autherID'] +'_'+ data['PID'].to_list(),data['label'].to_list()))
    t_df1['label'] = t_df1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)

    def get_sort_pid(a,b):
        ans = []
        for x,y in zip(a,b):
            if y == '':
                y =0
            ans.append([str(x),int(y)])

        ans =  sorted(ans,key = lambda x:x[1])

        res = []
        temp = []
        for x,y in ans:
            if len(temp) == 0 or y-temp[-1][1]<=1:
                temp.append([x,y])
            else:

                res.append( temp.copy())
                temp = []
                temp.append([x,y])
        if len(temp) != 0:
            res.append(temp.copy())

        return res

    t_df1['pid_split'] = t_df1.apply(lambda x:get_sort_pid(x['PID'],x['year']),axis = 1)

    ans = []
    for i in range(t_df1.shape[0]):

        tt = t_df1.loc[i]
        temp_pid_split = tt['pid_split'].copy()

        if len(temp_pid_split) <=1:
            tt['pid_split'] = temp_pid_split[0]
            ans.append(tt.copy().values)
        else:

            for xx in temp_pid_split:

                tt['pid_split'] = xx
                ans.append(tt.copy().values)

    t_df1_1 = pd.DataFrame(ans,columns = t_df1.columns)
    t_df1_1['PID'] = t_df1_1['pid_split'].apply(lambda x:[f[0] for f in x])
    t_df1_1['year'] = t_df1_1['pid_split'].apply(lambda x:[f[1] for f in x])
    t_df1_1 = t_df1_1[t_df1_1['year'].apply(lambda x:len(x)>1)]

    return t_df1_1

In [17]:
t_df_1_1 = get_split_year(t_df1)
t_df_2_1 = get_split_year(t_df2)
t_df_3_1 = get_split_year(t_df3.reset_index(drop = True))
t_df_4_1 = get_split_year(t_df4.reset_index(drop = True))

In [21]:
data['id'] = data.index

In [22]:
t_dict = dict(zip(data['autherID'] +'_'+ data['PID'].to_list(),data['id'].to_list()))
t_df_1_1['id'] = t_df_1_1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)
t_df_2_1['id'] = t_df_2_1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)
t_df_3_1['id'] = t_df_3_1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)
t_df_4_1['id'] = t_df_4_1.apply(lambda x:[t_dict[x['autherID'] + '_' +  f] for f in x['PID']],axis = 1)


In [23]:
def edge_engerate(x_list):
    res = []
    x_list = sorted(x_list)
    for i in range(len(x_list)):
        for j in range(i+1,len(x_list)):
            res.append((x_list[i],x_list[j]))
    
    return res

In [24]:
t_df_1_1['id'] = t_df_1_1['id'].map(edge_engerate)
edges_1_1 = t_df_1_1['id'].to_list()
edges_1_1 = np.concatenate(edges_1_1)



t_df_2_1['id'] = t_df_2_1['id'].map(edge_engerate)
edges_2_1 = t_df_2_1['id'].to_list()
edges_2_1 = np.concatenate(edges_2_1)


t_df_3_1['id'] = t_df_3_1['id'].map(edge_engerate)
edges_3_1 = t_df_3_1['id'].to_list()
edges_3_1 = np.concatenate(edges_3_1)


t_df_4_1['id'] = t_df_4_1['id'].map(edge_engerate)
edges_4_1 = t_df_4_1['id'].to_list()
edges_4_1 = np.concatenate(edges_4_1)

In [25]:
#edges = pd.concat([pd.DataFrame(edges_1_1),pd.DataFrame(edges_2),pd.DataFrame(edges_3),pd.DataFrame(edges_4)])
edges = pd.concat([pd.DataFrame(edges_1_1),pd.DataFrame(edges_2_1),pd.DataFrame(edges_3_1),pd.DataFrame(edges_4_1)])
#edges = pd.concat([pd.DataFrame(edges_4_1)])

#edges = pd.concat([pd.DataFrame(edges_1_1)])
edges = edges.drop_duplicates().reset_index(drop= True)
edges = edges.sort_values(0).reset_index(drop= True)

joblib.dump(edges,'edges_all.pkl')

['edges_all.pkl']

In [78]:
data = pd.read_feather('all_feat.feather')

In [79]:
all_feats = set(joblib.load('sub/feat_select_cat_feats.pkl'))| set(joblib.load('sub/feat_select_lgb_feats.pkl'))| set(joblib.load('sub/feat_select_xgb_feats.pkl')) | set(data.columns)


In [80]:
edges = joblib.load('edges_all.pkl')
data['id'] = data.index

In [81]:
edges_0 = dict(edges.groupby(0)[1].agg(list) )
edges_1 = dict(edges.groupby(1)[0].agg(list) )
df = data[['autherID','PID','label','id']]
df['id_0'] = df['id'].map(edges_0)
df['id_1'] = df['id'].map(edges_1)
df.loc[df['id_0'].isna(),'id_0']  ='[]'
df.loc[df['id_1'].isna(),'id_1']  ='[]'
df.loc[df['id_0'] == '[]','id_0']  =df.loc[df['id_0'] == '[]','id_0'].apply(lambda x:eval(x))
df.loc[df['id_1'] =='[]' ,'id_1']  =df.loc[df['id_1'] == '[]','id_1'].apply(lambda x:eval(x))
df['n_id'] = (df['id_0'] + df['id_1']).apply(lambda x:list(set(x)))
df['n_id'] = df['n_id'] + df['id'].apply(lambda x:[x])
df = df.drop(['id_0','id_1'],axis = 1)

In [82]:
group_feat =list(set(['_'.join(f.split('_')[2:-1]) for f in list(all_feats) if f.startswith('group_') and f != 'group_feat_cnt']))
for feat in tqdm(group_feat):
    t_dict = dict(data[['id',feat]].values)
    df[feat] = df['n_id'].apply(lambda x:[t_dict[f] for f in x])
    df[f'group_feat_{feat}_mean'] = df[feat].apply(lambda x:np.mean(x))
    df[f'group_feat_{feat}_std'] = df[feat].apply(lambda x:np.std(x))
    df[f'group_feat_{feat}_cnt'] = df[feat].apply(lambda x:len(x))
    del df[feat]
df[f'group_feat_cnt'] = df['n_id'].apply(lambda x:len(x))
df  =df.drop(['id','n_id','label'],axis = 1)

100%|██████████| 34/34 [00:04<00:00,  8.15it/s]


In [83]:

data = data.merge(df,on = ['autherID','PID'],how = 'left')

In [84]:
data.to_feather('all_feats2.feather')